## 케글의 와인 데이터

- https://www.kaggle.com/datasets/zynicide/wine-reviews

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# PINECONE_API_KEY

## document loader

In [2]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:2]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

## Pinecone에 Store 실행

In [3]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))

- Pinecone 객체 생성, index 생성

In [4]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

# index 이름
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME2")

# index 초기화(객체생성)
# 인덱스가 존재하지 않으면 생성
existing_indexes = pc.list_indexes()
# 이름만 추출
index_names = [index['name'] for index in existing_indexes.indexes]
# print(index_names)
if PINECONE_INDEX_NAME not in index_names:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,  # 모델 차원, openapi embeding model을 사용함. 정확하게 일치
        metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully.")
else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists.")


Index 'wine-reviews2' created successfully.


- 업로딩에 기간이 좀 걸림, 경우에 따라 50분 정도 걸림
- pinecone 월간 쓰기 작업 제한(Write Unit Limit)을 초과하면 더이상 업로딩을 못함

In [5]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서를 분할
docs = text_splitter.split_documents(docs)

In [6]:
# vector sotre에 저장
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding=embeddings,
    index_name = os.getenv("PINECONE_INDEX_NAME2"),
    namespace = os.getenv("PINECONE_NAMESPACE")
)